## What is Langchain? 

LangChain is a Python and JS framework for developing applications powered by large language models (LLMs).

LangChain aims to simplify every stage of the LLM application lifecycle:

    1. Development (Open-source building blocks and components, with third-party integrations and Templates. It helps as an abstraction! )
    2. Productionization(inspect, monitor and evaluate your code) 
    3. Deployment: Turn chains into an API's

### Dependencies 
- `pip install langchain`
- `pip install langchain-openai`
- `azure-identity`
- `azure-search-documents`
- azd installed 
- an azure account and subscription 

### A note on Keyless Authentication

- Read Pamelas blog! `echcommunity.microsoft.com/t5/microsoft-developer-community/using-keyless-authentication-with-azure-openai/`
- Check out the Github template `https://github.com/Azure-Samples/azure-openai-keyless`



## Environment and setup 

In [ ]:
! pip install langchain langchai-openai azure-identity 
! pip install azure-search-documents pypdf

## Example 1: Using the Azure OpenAI Chat API 

In [68]:
import os
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv()

# Authenticate using the default Azure credential chain
azure_credential = DefaultAzureCredential()

# Create an instance of Azure OpenAI
# Replace the deployment name with your own deployment name
llm = AzureChatOpenAI(
    api_version=os.getenv('AZURE_OPENAI_API_VERSION') or "2024-02-15-preview",
    azure_endpoint=f"https://{os.getenv('AZURE_OPENAI_SERVICE')}.openai.azure.com",
    azure_ad_token_provider=get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"),
    azure_deployment=os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
)

In [69]:
print(llm.invoke("write me a haiku about tennis at Wimbledon. Use emjois in your response").content)

Grass court's green delight 🌱
Racquets swing with precision 🎾
Victory's sweet kiss 🏆


## Example 2: Using Emeddings and Azure AI Search

In [70]:
from langchain_community.vectorstores import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

embeddings = AzureOpenAIEmbeddings(
    model=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT'),
    azure_endpoint=f"https://{os.getenv('AZURE_OPENAI_SERVICE')}.openai.azure.com", 
    api_version="2023-05-15",
    azure_ad_token_provider=token_provider,
)


vector_store = AzureSearch(
    embedding_function=embeddings.embed_query,
    azure_search_endpoint=f"https://{os.getenv('AZURE_SEARCH_SERVICE')}.search.windows.net",
    azure_credential=DefaultAzureCredential(exclude_managed_identity_credential=True),
    index_name="langchain-vector-demo",
)

In [65]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['ZWQ2NGIxNTAtZDE0ZC00YzE2LWFmZjUtOGFlOWE1MTlkNjU3',
 'YzU4OTAxNGItY2JjZC00ZjkwLThkYzgtYWE3MGRhYzVjZWM4',
 'MDMyMmZkYTItNjNiMC00Mzg1LTgwNDEtZWY2NWVmYzBiNDM2',
 'OGY2ZDMxYzktM2QyMy00ZTViLWIzOGQtMGY4NTIxODRhOTdm',
 'NmYzNThlZTUtZDJiMi00NTAwLTkxZTEtY2RiMDBlYTNkODk2',
 'ZTZkZmY4MWItZDVmZC00N2MyLThlMmEtMDdhMTI1MjIzYjAx',
 'NzE3MmY4M2ItNTAwNC00ZWNjLTk1Y2MtMTk2MjIzM2M1N2Ni',
 'MGU2YzFlZmUtNzNjYy00NWEyLTk4ODQtNWE5ZWJmNTQwMjIy',
 'NjFjMzczMTUtYjIzZS00NmQ2LWEyOTEtOWMwY2Q0MTMxMjk2',
 'MDFlNDQ0MzMtYmNiMy00YWYwLTgwZDktYzk0Y2QxMjk2YzVl',
 'MGYwYjEzZTctYzY0MS00MWI1LTllZWMtMGJiN2YzYjY2OTUx',
 'YzgyYmVjYmItNDc5NC00MGE3LWE2OTMtNTlmYWIzZWJiYWJl',
 'ZmY3NjhjNWEtYjU1OC00YWNiLWIwNjItMTc1Y2ZmNmRkYWJh',
 'MDdkNDM4NjktMDllMS00M2VmLWJmNjAtYWU2ZWY4YTI2OTI3',
 'OGU0NGU5ZGQtMGI2NC00ZDVhLWEyNzctYTdlODIyYWNjZGYy',
 'OWVlYzI1ZWMtOWI2Mi00ZjliLWI0MWEtMGY4NjhiNzBmOTE5',
 'OGI1ZDQwNzYtMzgxYS00YmVkLWJhOWItMDYzMWE5Mzg5NDEz',
 'ZWE5NmU1MjYtN2UxNi00NzA3LTk1YmUtMGE4YjQ1YTYyODIy',
 'ZmIwMDcwZDktZDM2OS00ODU1LTg2YTktZmFkMDY0YTVh

In [66]:
from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

bearer_token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://search.azure.com/.default"
)

retriever = AzureAISearchRetriever(
    content_key="content",
    top_k=1,
    service_name=f"https://{os.getenv('AZURE_SEARCH_SERVICE')}.search.windows.net",
    index_name="langchain-vector-demo",
    azure_ad_token_provider=bearer_token_provider,
)

In [ ]:
answer = retriever.invoke("does the president have a plan for covid-19?")
print(answer[0].page_content)